In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [2]:
sentence = (" if you want to build a ship, don't drum up people together to "
          "collect wood and don't assign them tasks and work, but rather "
          "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10

dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i+1:i + seq_length +1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)

0  if you wa -> if you wan
1 if you wan -> f you want
2 f you want ->  you want 
3  you want  -> you want t
4 you want t -> ou want to
5 ou want to -> u want to 
6 u want to  ->  want to b
7  want to b -> want to bu
8 want to bu -> ant to bui
9 ant to bui -> nt to buil
10 nt to buil -> t to build
11 t to build ->  to build 
12  to build  -> to build a
13 to build a -> o build a 
14 o build a  ->  build a s
15  build a s -> build a sh
16 build a sh -> uild a shi
17 uild a shi -> ild a ship
18 ild a ship -> ld a ship,
19 ld a ship, -> d a ship, 
20 d a ship,  ->  a ship, d
21  a ship, d -> a ship, do
22 a ship, do ->  ship, don
23  ship, don -> ship, don'
24 ship, don' -> hip, don't
25 hip, don't -> ip, don't 
26 ip, don't  -> p, don't d
27 p, don't d -> , don't dr
28 , don't dr ->  don't dru
29  don't dru -> don't drum
30 don't drum -> on't drum 
31 on't drum  -> n't drum u
32 n't drum u -> 't drum up
33 't drum up -> t drum up 
34 t drum up  ->  drum up p
35  drum up p -> drum up pe
36

In [3]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell]*2, state_is_tuple=True)

outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [4]:
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])

softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])

outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])

In [5]:
weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)

train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run([train_op, mean_loss, outputs],
                             feed_dict={X: dataX, Y: dataY})
    
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

0 0 oooooooooo 3.26563
0 1 oooooooooo 3.26563
0 2 osssoooooo 3.26563
0 3 oooooooooo 3.26563
0 4 oooooooooo 3.26563
0 5 oooooooooo 3.26563
0 6 oooooooooo 3.26563
0 7 oooooooooo 3.26563
0 8 oooooooooo 3.26563
0 9 oooooooooo 3.26563
0 10 oooooooooo 3.26563
0 11 oooooooooo 3.26563
0 12 oooooooooo 3.26563
0 13 oooooooooo 3.26563
0 14 oooooooooo 3.26563
0 15 oooooooooo 3.26563
0 16 ooomoooooo 3.26563
0 17 oooooooooo 3.26563
0 18 oooooooooo 3.26563
0 19 oooooooooo 3.26563
0 20 oooooooooo 3.26563
0 21 oooooooooo 3.26563
0 22 oooooooooo 3.26563
0 23 oooooooooo 3.26563
0 24 oooooooooo 3.26563
0 25 oooooooooo 3.26563
0 26 oooooooooo 3.26563
0 27 oooooooooo 3.26563
0 28 oooooooooo 3.26563
0 29 oooooooooo 3.26563
0 30 oooooooooo 3.26563
0 31 oooooooooo 3.26563
0 32 oooooooooo 3.26563
0 33 oooooooooo 3.26563
0 34 oooooooooo 3.26563
0 35 oooooooooo 3.26563
0 36 oooooooooo 3.26563
0 37 oossoooooo 3.26563
0 38 oooooooooo 3.26563
0 39 oooooooooo 3.26563
0 40 oooooooooo 3.26563
0 41 oooooooooo 3.26563
0 

3 0 ottttttttt 2.91162
3 1 ootttttttt 2.91162
3 2 ootttttttt 2.91162
3 3 ottttttttt 2.91162
3 4 ottttttttt 2.91162
3 5 ottttttttt 2.91162
3 6 ottttttttt 2.91162
3 7 ottttttttt 2.91162
3 8 ottttttttt 2.91162
3 9 ootttttttt 2.91162
3 10 ootttttttt 2.91162
3 11 ootttttttt 2.91162
3 12 ottttttttt 2.91162
3 13 ootttttttt 2.91162
3 14 ottttttttt 2.91162
3 15 ottttttttt 2.91162
3 16 ottttttttt 2.91162
3 17 ottttttttt 2.91162
3 18 ootttttttt 2.91162
3 19 ooottttttt 2.91162
3 20 ottttttttt 2.91162
3 21 ottttttttt 2.91162
3 22 ottttttttt 2.91162
3 23 ootttttttt 2.91162
3 24 ooottttttt 2.91162
3 25 ootttttttt 2.91162
3 26 ootttttttt 2.91162
3 27 ooottttttt 2.91162
3 28 ootttttttt 2.91162
3 29 ottttttttt 2.91162
3 30 ottttttttt 2.91162
3 31 ottttttttt 2.91162
3 32 ootttttttt 2.91162
3 33 ooottttttt 2.91162
3 34 ootttttttt 2.91162
3 35 ottttttttt 2.91162
3 36 ottttttttt 2.91162
3 37 ottttttttt 2.91162
3 38 ottttttttt 2.91162
3 39 ooottttttt 2.91162
3 40 ottttttttt 2.91162
3 41 ottttttttt 2.91162
3 

5 0 eetttttt   2.88233
5 1 eetttttt   2.88233
5 2 ottttttt   2.88233
5 3 etttttt    2.88233
5 4 ettttttt   2.88233
5 5 etttttttt  2.88233
5 6 ettttttt   2.88233
5 7 eettttt    2.88233
5 8 etttttt    2.88233
5 9 oettt      2.88233
5 10 eetttt     2.88233
5 11 eetttt     2.88233
5 12 eettttt    2.88233
5 13 eettttt    2.88233
5 14 ettttttt   2.88233
5 15 etttttt    2.88233
5 16 etttttt    2.88233
5 17 etttttt    2.88233
5 18 eeet       2.88233
5 19 oettt      2.88233
5 20 otttttttt  2.88233
5 21 ettttttt   2.88233
5 22 oettttt    2.88233
5 23 eetttt     2.88233
5 24 oeee       2.88233
5 25 eetttt     2.88233
5 26 eetttt     2.88233
5 27 ettttt     2.88233
5 28 otttttt    2.88233
5 29 etttttt    2.88233
5 30 otttttt    2.88233
5 31 ettttt     2.88233
5 32 eeett      2.88233
5 33 eeee       2.88233
5 34 eetttt     2.88233
5 35 ettttttttt 2.88233
5 36 ottttttttt 2.88233
5 37 etttttttt  2.88233
5 38 ettttttt   2.88233
5 39 oetttt     2.88233
5 40 ettttttt   2.88233
5 41 ettttttt   2.88233
5 

7 119 o          2.85318
7 120 o          2.85318
7 121 e          2.85318
7 122 o          2.85318
7 123 e          2.85318
7 124 o          2.85318
7 125 o          2.85318
7 126 o          2.85318
7 127 o          2.85318
7 128 o          2.85318
7 129 e          2.85318
7 130 o          2.85318
7 131 o          2.85318
7 132 e          2.85318
7 133 o          2.85318
7 134 o          2.85318
7 135 o          2.85318
7 136 o          2.85318
7 137 e          2.85318
7 138 o          2.85318
7 139 o          2.85318
7 140 e          2.85318
7 141 o          2.85318
7 142 e          2.85318
7 143 o          2.85318
7 144 o          2.85318
7 145 e          2.85318
7 146 e          2.85318
7 147 o          2.85318
7 148 o          2.85318
7 149 e          2.85318
7 150 o          2.85318
7 151 o          2.85318
7 152 o          2.85318
7 153 o          2.85318
7 154 o          2.85318
7 155 o          2.85318
7 156 o          2.85318
7 157            2.85318
7 158            2.85318


10 77            2.83283
10 78 t          2.83283
10 79            2.83283
10 80 t          2.83283
10 81            2.83283
10 82            2.83283
10 83            2.83283
10 84            2.83283
10 85            2.83283
10 86            2.83283
10 87            2.83283
10 88            2.83283
10 89            2.83283
10 90            2.83283
10 91            2.83283
10 92            2.83283
10 93            2.83283
10 94            2.83283
10 95            2.83283
10 96            2.83283
10 97            2.83283
10 98            2.83283
10 99            2.83283
10 100            2.83283
10 101            2.83283
10 102            2.83283
10 103            2.83283
10 104            2.83283
10 105            2.83283
10 106 t          2.83283
10 107            2.83283
10 108            2.83283
10 109            2.83283
10 110            2.83283
10 111            2.83283
10 112            2.83283
10 113            2.83283
10 114            2.83283
10 115 t          2.83283
10 116   

...

488 0 tf you wan 0.230429
488 1 f you want 0.230429
488 2 oyou want  0.230429
488 3 tou want t 0.230429
488 4 ou want to 0.230429
488 5 n want to  0.230429
488 6 iwant to b 0.230429
488 7 tont to bu 0.230429
488 8 ont to bui 0.230429
488 9 nd to buil 0.230429
488 10 d to build 0.230429
488 11 hro build  0.230429
488 12 to cuild a 0.230429
488 13 h luild a  0.230429
488 14 nbuild a s 0.230429
488 15 tuild a sh 0.230429
488 16 uild a shi 0.230429
488 17 ild a ship 0.230429
488 18 fd a ship, 0.230429
488 19 e a ship,  0.230429
488 20  anship, d 0.230429
488 21 tnship, do 0.230429
488 22 nship, don 0.230429
488 23 thip, don' 0.230429
488 24  ip, don't 0.230429
488 25 ep, don't  0.230429
488 26 f, don't d 0.230429
488 27   don't dr 0.230429
488 28  bon't dru 0.230429
488 29 ton't drum 0.230429
488 30  n't arum  0.230429
488 31 n't arum u 0.230429
488 32 dt arum up 0.230429
488 33 t arum up  0.230429
488 34 hrrum up p 0.230429
488 35 toum up pe 0.230429
488 36  um up peo 0.230429
488 37  m u

490 153 d ess imme 0.23044
490 154  ess immen 0.23044
490 155 ecs immens 0.23044
490 156 rs immensi 0.23044
490 157 i immensit 0.23044
490 158 iimmensity 0.23044
490 159 tmmensity  0.23044
490 160 fmensity o 0.23044
490 161  ensity of 0.23044
490 162  nsity of  0.23044
490 163 rsity of t 0.23044
490 164 dity of th 0.23044
490 165 ity of the 0.23044
490 166 fy of the  0.23044
490 167 h of the s 0.23044
490 168  of the se 0.23044
490 169 tf the sea 0.23044
490 170 n the sea. 0.23044
491 0 tf you wan 0.230485
491 1 p you want 0.230485
491 2 oyou want  0.230485
491 3 tou want t 0.230485
491 4 ou want to 0.230485
491 5 n want to  0.230485
491 6  want to b 0.230485
491 7 tont to bu 0.230485
491 8 ont to bui 0.230485
491 9 nd to buil 0.230485
491 10 d to build 0.230485
491 11  ro build  0.230485
491 12 to cuild a 0.230485
491 13   luild a  0.230485
491 14 nbuild a s 0.230485
491 15 tuild a sh 0.230485
491 16 uild a shi 0.230485
491 17  ld a ship 0.230485
491 18 pd a ship, 0.230485
491 19 e a 

493 117 tather tea 0.230328
493 118  ther teac 0.230328
493 119 nher teach 0.230328
493 120 hem toach  0.230328
493 121 em teach t 0.230328
493 122 r teach th 0.230328
493 123  thach the 0.230328
493 124 toach them 0.230328
493 125 hach them  0.230328
493 126 rch them t 0.230328
493 127 nh them to 0.230328
493 128 h them to  0.230328
493 129 ethem to l 0.230328
493 130 toem ta lo 0.230328
493 131 hem ta lon 0.230328
493 132 em ta long 0.230328
493 133 r to long  0.230328
493 134  ta long f 0.230328
493 135 to cong fo 0.230328
493 136 h long for 0.230328
493 137  cong for  0.230328
493 138 tong for t 0.230328
493 139 eng for th 0.230328
493 140  ' for the 0.230328
493 141 d for the  0.230328
493 142 efor the e 0.230328
493 143 tor the en 0.230328
493 144  r the end 0.230328
493 145   the endl 0.230328
493 146  the endle 0.230328
493 147 toemendles 0.230328
493 148 hemendless 0.230328
493 149 emendless  0.230328
493 150 rtndless i 0.230328
493 151 tndless im 0.230328
493 152 rdless imm 0

496 104 nd work, b 0.230321
496 105 d work, bu 0.230321
496 106  aork, but 0.230321
496 107 tork, but  0.230321
496 108 ork, but r 0.230321
496 109 n , but ra 0.230321
496 110  , but rat 0.230321
496 111 s but rath 0.230321
496 112  dut rathe 0.230321
496 113 tut rather 0.230321
496 114 ut rather  0.230321
496 115 p rather t 0.230321
496 116 hwather te 0.230321
496 117 tather tea 0.230321
496 118  ther teac 0.230321
496 119 nher teach 0.230321
496 120 her toach  0.230321
496 121 er teach t 0.230321
496 122   teach th 0.230321
496 123  teach the 0.230321
496 124 toach them 0.230321
496 125 hach them  0.230321
496 126  ch them t 0.230321
496 127 nh them to 0.230321
496 128 t them to  0.230321
496 129 ethem to l 0.230321
496 130 toem to lo 0.230321
496 131 her to lon 0.230321
496 132 er ta long 0.230321
496 133   to long  0.230321
496 134  ta long f 0.230321
496 135 to cong fo 0.230321
496 136 h long for 0.230321
496 137 nlong for  0.230321
496 138 tong for t 0.230321
496 139 eng for th 0

499 91 dthem task 0.230321
499 92 toem tosks 0.230321
499 93  er tosks  0.230321
499 94 er tosks a 0.230321
499 95   tosks an 0.230321
499 96  tosks and 0.230321
499 97 tosks and  0.230321
499 98  sks and w 0.230321
499 99 nks and wo 0.230321
499 100  s and wor 0.230321
499 101 , and work 0.230321
499 102  ind work, 0.230321
499 103 tnd dork,  0.230321
499 104 nd work, b 0.230321
499 105 d work, bu 0.230321
499 106  aork, but 0.230321
499 107 took, but  0.230321
499 108 ook, but r 0.230321
499 109 nk, but ra 0.230321
499 110  , but rat 0.230321
499 111 , but rath 0.230321
499 112  dut rathe 0.230321
499 113 tut rather 0.230321
499 114 ut rather  0.230321
499 115   rather t 0.230321
499 116  rather te 0.230321
499 117 tather tea 0.230321
499 118  ther teac 0.230321
499 119 nher teach 0.230321
499 120  er teach  0.230321
499 121 er teach t 0.230321
499 122   teach th 0.230321
499 123  teach the 0.230321
499 124 toach them 0.230321
499 125  ach them  0.230321
499 126  ch them t 0.230321
4

In [8]:
results = sess.run(outputs, feed_dict={X:dataX})

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

tf you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.